In [ ]:
import os
import json
from dotenv import load_dotenv
import pandas as pd
import copy
import re
import time

from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage



In [12]:
# -----------------------------
#  Load Environment Variables
# -----------------------------
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")

if not openai_key:
    raise ValueError("❌ OPENAI_API_KEY not found in .env file")


# -----------------------------
#  Initialize LLM (GPT-4o-mini)
# -----------------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, api_key=openai_key)
response = llm.invoke([HumanMessage(content="Generate a creative name for a toy car")])
print(response.content)

"Zoominator"


In [13]:

# -----------------------------
#  Define Agent State
# -----------------------------
class AgentState(dict):
    """Holds intermediate state during graph execution."""
    pass



# -----------------------------
#  Node: Load Excel
# -----------------------------
def load_excel(state: AgentState):
    df = pd.read_excel(state["input_path"])
    state["data"] = df
    return state



In [14]:
# Loading Input data:
input_data = f"data_cap.xlsx"
og_df = pd.read_excel(input_data)
print(f"Original data loaded: {len(og_df)} rows")

# Create a deep copy for transformations:
df = copy.deepcopy(og_df)
df.head()


Original data loaded: 73100 rows


,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn
1,2022-01-01,S001,P0002,Toys,South,204,150,66,144.04,63.01,20,Sunny,0,66.16,Autumn
2,2022-01-01,S001,P0003,Toys,West,102,65,51,74.02,27.99,10,Sunny,1,31.32,Summer
3,2022-01-01,S001,P0004,Toys,North,469,61,164,62.18,32.72,10,Cloudy,1,34.74,Autumn
4,2022-01-01,S001,P0005,Electronics,East,166,14,135,9.26,73.64,0,Sunny,0,68.95,Summer


## Updating Product IDs:

In [15]:

# -----------------------------------------------
#  Utility: Get Unique Categories
# -----------------------------------------------

def get_unique_categories(df, category_col="Category"):
    if category_col not in df.columns:
        raise ValueError(f"Column '{category_col}' not found in DataFrame.")
    unique_cats = sorted(df[category_col].dropna().unique().tolist())
    print(f"Found {len(unique_cats)} unique categories: {unique_cats}")
    return unique_cats



# -----------------------------------------------
#  Utility: Create Prefix Mapping
# -----------------------------------------------

def create_prefix_mapping(categories):
    mapping = {}
    used = set()

    for cat in categories:
        prefix = cat[0].upper()
        # Handle duplicate starting letters:
        if prefix in used:
            for ch in cat[1:].upper():
                if ch not in used and ch.isalpha():
                    prefix = ch
                    break
        used.add(prefix)
        mapping[cat] = prefix

    print("Category → Prefix mapping:")
    for k, v in mapping.items():
        print(f"   {k:15s} → {v}")
    return mapping


def save_prefix_mapping(mapping, path="category_prefix_map.json"):
    with open(path, "w") as f:
        json.dump(mapping, f, indent=4)
    print(f"Prefix mapping saved to {path}")


def load_prefix_mapping(path="category_prefix_map.json"):
    with open(path, "r") as f:
        mapping = json.load(f)
    print(f"Loaded prefix mapping from {path}")
    return mapping



In [16]:
# Creating the productid prefix map:
categories = get_unique_categories(df)
prefix_map = create_prefix_mapping(categories)


Found 5 unique categories: ['Clothing', 'Electronics', 'Furniture', 'Groceries', 'Toys']
Category → Prefix mapping:
   Clothing        → C
   Electronics     → E
   Furniture       → F
   Groceries       → G
   Toys            → T


In [17]:
# Mapping JSON path:
mapping_json_name = "category_prefix_map"
mapping_json_path = f"{mapping_json_name}.json"

# # Saving Mapping JSON:
# save_prefix_mapping(prefix_map, path=mapping_json_path)

# Loading Mapping JSON:
json_prefix_map = load_prefix_mapping(mapping_json_path)
json_prefix_map


Loaded prefix mapping from category_prefix_map.json


{'Clothing': 'C',
 'Electronics': 'E',
 'Furniture': 'F',
 'Groceries': 'G',
 'Toys': 'T'}

In [18]:

# -----------------------------------------------
#  Utility: Saving df to xlsx
# -----------------------------------------------

def save_dataframe_to_xlsx(df, base_filename):
    output_path = f"{base_filename}.xlsx"
    df.to_excel(output_path, index=False)
    print(f"File saved successfully → {output_path}")
    return output_path



# -----------------------------------------------
#  Utility: Updating Product Ids
# -----------------------------------------------

def update_product_ids_in_df(df, prefix_mapping, output_path="input_data_with_new_pids"):

    # Generate new product IDs directly from df:
    def generate_new_id(row):
        old_id = str(row["Product ID"])
        category = row["Category"]
        prefix = prefix_mapping.get(category, "X")  # Default 'X' if not found
        return prefix + old_id[1:] if old_id else None

    # Create new column:
    df["New Product ID"] = df.apply(generate_new_id, axis=1)

    # Update original Product ID column:
    df["Product ID"] = df["New Product ID"]

    # Save updated DataFrame as new Excel:
    save_dataframe_to_xlsx(df, output_path)
    print(f"Product IDs updated and saved to {output_path}.xlsx")
    return df




In [19]:
# New data path:
new_data_name = "input_data_with_new_pids"
new_data_path = f"{new_data_name}"


# Update directly using the in-memory df:
updated_df = update_product_ids_in_df(df, prefix_map, output_path=new_data_path)


File saved successfully → input_data_with_new_pids.xlsx
Product IDs updated and saved to input_data_with_new_pids.xlsx


In [20]:
updated_df.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality,New Product ID
0,2022-01-01,S001,G0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn,G0001
1,2022-01-01,S001,T0002,Toys,South,204,150,66,144.04,63.01,20,Sunny,0,66.16,Autumn,T0002
2,2022-01-01,S001,T0003,Toys,West,102,65,51,74.02,27.99,10,Sunny,1,31.32,Summer,T0003
3,2022-01-01,S001,T0004,Toys,North,469,61,164,62.18,32.72,10,Cloudy,1,34.74,Autumn,T0004
4,2022-01-01,S001,E0005,Electronics,East,166,14,135,9.26,73.64,0,Sunny,0,68.95,Summer,E0005


## Product name creation:

In [54]:
# Directly load the updated productid xlsx:
new_pid_data = f"input_data_with_new_pids.xlsx"
new_pid_df = pd.read_excel(new_pid_data)
print(f"new_pid_df data loaded: {len(new_pid_df)} rows")


# Create a deep copy for transformations:
df = copy.deepcopy(new_pid_df)
df.head()



new_pid_df data loaded: 73100 rows


,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality,New Product ID
0,2022-01-01,S001,G0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn,G0001
1,2022-01-01,S001,T0002,Toys,South,204,150,66,144.04,63.01,20,Sunny,0,66.16,Autumn,T0002
2,2022-01-01,S001,T0003,Toys,West,102,65,51,74.02,27.99,10,Sunny,1,31.32,Summer,T0003
3,2022-01-01,S001,T0004,Toys,North,469,61,164,62.18,32.72,10,Cloudy,1,34.74,Autumn,T0004
4,2022-01-01,S001,E0005,Electronics,East,166,14,135,9.26,73.64,0,Sunny,0,68.95,Summer,E0005


In [ ]:
# ------------------------------------
#  Node: Gather Product specific rows
# ------------------------------------

# def gather_product_data(df, product_id):
#     sub_df = df[df["Product ID"] == product_id]
#     summary_lines = []
#     for _, row in sub_df.iterrows():
#         line = (
#             f"- Category: {row['Category']}, Region: {row['Region']}, "
#             f"Price: {row['Price']}, Discount: {row['Discount']}, "
#             f"Weather: {row['Weather Condition']}, Season: {row['Seasonality']}, "
#             f"Holiday/Promotion: {row['Holiday/Promotion']}, Demand Forecast: {row['Demand Forecast']}"
#         )
#         summary_lines.append(line)
    
#     summary_text = "\n".join(summary_lines)
#     return summary_text



def gather_product_data(df, product_id):
    """
    Summarizes the overall seasonal, weather, and holiday context 
    for a product by lightly aggregating its data across all stores and months.
    Returns a compact descriptive string.
    """
    # Normalize product ID
    df["Product ID"] = df["Product ID"].astype(str).str.strip().str.upper()
    product_id = str(product_id).strip().upper()

    # Filter for this product
    sub_df = df[df["Product ID"] == product_id].copy()
    if sub_df.empty:
        return ""

    # Get key context values
    category = sub_df["Category"].iloc[0]
    avg_price = round(sub_df["Price"].mean(), 2)
    avg_discount = round(sub_df["Discount"].mean(), 2)
    common_weather = sub_df["Weather Condition"].mode()[0]
    common_season = sub_df["Seasonality"].mode()[0]
    holiday_count = sub_df["Holiday/Promotion"].sum()
    regions = ", ".join(sub_df["Region"].unique())

    # Create a compact context string
    context = (
        f"Category: {category}, Avg Price: {avg_price}, Avg Discount: {avg_discount}%, "
        f"Common Weather: {common_weather}, Common Season: {common_season}, "
        f"Holiday/Promotion Frequency: {holiday_count}, Active Regions: {regions}."
    )
    return context




In [ ]:


# -----------------------------
#  Node: Generate Product Names
# -----------------------------
# def generate_name(row):
#     prompt = f"""
#     You are a creative branding expert.

#     Create a **full product name** that includes:
#     1. What the item actually is (e.g., "wooden chair", "toy car", "LED lamp").
#     2. A creative or premium-sounding brand-style name.
#     3. Any relevant regional, weather, or seasonal context.
#     4. Holiday or promotion hints if applicable.

#     Data:
#     - Product ID: {row['Product ID']}
#     - Category: {row['Category']}
#     - Region: {row['Region']}
#     - Price: {row['Price']}
#     - Discount: {row['Discount']}
#     - Weather: {row['Weather Condition']}
#     - Season: {row['Seasonality']}
#     - Holiday/Promotion: {row['Holiday/Promotion']}

#     Output only the product name (no explanation).
#     """
#     result = llm.invoke([
#         SystemMessage(content="You are a creative marketing and branding assistant."),
#         HumanMessage(content=prompt)
#     ])
#     return result.content.strip()




def generate_name_for_product(product_id, category, product_data):
    """
    Generates a complete, market-ready product name that uniquely identifies
    a tangible product (not just its category), using all aggregated data.
    """
    prompt = f"""
    You are an expert product naming specialist working for a global retail brand.

    You will generate a **unique, object-specific product name** for Product ID {product_id}.
    The goal is to make it sound like a real, purchasable item — not a generic category or set.

    **Product Context:**
    {product_data}

    Create a **realistic, distinctive, and descriptive product name** that:
    1. Clearly identifies what the product actually is — a tangible object a person can buy,
       such as "wooden chess board", "foldable camping chair", "aroma diffuser lamp".
       Avoid generic classes like "set", "groceries", "furniture", or "electronics".
    2. Includes a creative or premium-sounding brand prefix, descriptor, or style element 
       (e.g., “AuroraCraft”, “LuxeGlow”, “HarvestBloom”).
    3. Optionally reflects the category "{category}" and contextual cues
       (season, weather, holidays, promotions, or regional traits).
    4. Feels authentic to modern branding — professional, elegant, concise, and memorable.
    5. Is **2–5 words long** and sounds like something seen in an online store or catalog.
    6. Avoid any vague or collective names ("set", "bundle", "pack", "groceries") unless it is a specific item name.
    7. Make the name sound **unique** — as if it belongs to one particular SKU or product variant.

    Output only the final product name — no explanations, no punctuation, no quotes.
    """

    result = llm.invoke([
        SystemMessage(content="You are a creative branding and product naming specialist."),
        HumanMessage(content=prompt)
    ])
    return result.content.strip()




In [ ]:

# -------------------------------------
#  Node: Reflection for product name
# -------------------------------------
# def reflect_name(name, row, pass_num=1):
#     prompt = f"""
#     Reflection Pass {pass_num}:

#     The current product name is: "{name}"

#     Evaluate and improve it for:
#     - Creativity and emotional appeal
#     - Clarity of what the item actually is
#     - Fit for its category: {row['Category']}
#     - Relevance to region: {row['Region']}
#     - Price and discount context (affordable vs premium)
#     - Weather: {row['Weather Condition']}
#     - Seasonal relevance: {row['Seasonality']}
#     - Holiday or promotion theme: {row['Holiday/Promotion']}

#     Suggest ONE improved version (keep it realistic and marketable).
#     Output only the improved product name.
#     """
#     response = llm.invoke([HumanMessage(content=prompt)])
#     return response.content.strip()


def reflect_product_name(product_id, name, product_data, category, pass_num=1):
    """
    Refines and improves a previously generated product name,
    ensuring it describes a specific, tangible product rather than a generic type.
    """
    prompt = f"""
    Reflection Pass {pass_num}:

    Product ID: {product_id}
    Category: {category}
    Current product name: "{name}"

    **Product Context:**
    {product_data}

    Reflect on and refine this product name.

    Your goal is to improve it so that it:
    1. Describes a **specific physical product**, not a general type or collection.
       For example, "AuroraCraft Wooden Chess Board" is valid, but "Chess Set" or "Groceries" is not.
    2. Keeps brand realism — something that could appear in a store listing.
    3. Uses 2–5 words with a clear brand-style prefix, material, or feature (e.g., "Luxe", "Glow", "Forge", "Pure", "Flex").
    4. Incorporates contextual cues — seasonal, promotional, or regional if relevant — subtly, not explicitly.
    5. Sounds elegant, marketable, and memorable.
    6. Avoids vague, plural, or abstract phrasing ("set", "groceries", "furniture", "electronics").
    7. Stays concise, professional, and polished.

    Suggest ONE improved version that fulfills these conditions.

    Output only the final improved product name — no explanations or punctuation.
    """

    response = llm.invoke([HumanMessage(content=prompt)])
    return response.content.strip()


In [ ]:

# -----------------------------------------------
#  Node: Reflection pipeline for multiple pass
# -----------------------------------------------

# def multiple_pass_reflection(row, num_passes=2):
#     name = generate_name(row)
#     print("Initial:", name)
#     for i in range(num_passes):
#         name = reflect_name(name, row, pass_num=i+1)
#         print(f"Reflection {i+1}:", name)
#     return name


def multiple_pass_reflection_by_product(df, num_passes=2, product_name_map=None, force_regenerate=False):
    """
    Generates and reflects product names for unique Product IDs,
    maintaining a persistent map of Product ID → Final Name.

    If a product already exists in the map and `force_regenerate=True`,
    or its cached name is empty/invalid, the name is recreated.
    """
    results = []
    unique_products = df["Product ID"].astype(str).str.strip().str.upper().unique()
    print(f"🧩 Found {len(unique_products)} unique products")

    # Initialize the cache if not provided
    if product_name_map is None:
        product_name_map = {}

    for pid in unique_products:
        pid_key = str(pid).strip().upper()
        category = df.loc[df["Product ID"].astype(str).str.strip().str.upper() == pid_key, "Category"].iloc[0]

        # Check if name already exists and is valid
        existing_name = product_name_map.get(pid_key, "").strip()
        should_regenerate = (
            force_regenerate or not existing_name or existing_name.lower() in ["", "nan", "none"]
        )

        if not should_regenerate:
            print(f"⚙️ Skipping {pid_key} — existing name: {existing_name}")
            results.append({
                "Product ID": pid_key,
                "Category": category,
                f"RefinedName_{num_passes}": existing_name
            })
            continue

        # 🧠 (Re)generate new name
        print(f"\n🔹 Processing Product ID: {pid_key} (Regenerate: {should_regenerate})")

        product_data = gather_product_data(df, pid_key)
        print(f"\n📘 product_data for {pid_key}:\n{product_data}\n")

        # Generate base name
        name = generate_name_for_product(pid_key, category, product_data)
        print("🪄 Initial Name:", name)

        # Multi-pass reflection
        for i in range(num_passes):
            name = reflect_product_name(pid_key, name, product_data, category, pass_num=i+1)
            print(f"💬 Reflection {i+1}:", name)

        final_name = name.strip()
        product_name_map[pid_key] = final_name  # ✅ Update the cache

        results.append({
            "Product ID": pid_key,
            "Category": category,
            f"RefinedName_{num_passes}": final_name
        })

    print(f"\n✅ Completed. Finalized names for {len(product_name_map)} products.\n")
    return pd.DataFrame(results), product_name_map


In [ ]:

# -----------------------------------------------
#  Utility: Updating product names
# -----------------------------------------------


def assign_product_names_to_main_df(main_df, product_names_df):
    """
    Directly updates the main DataFrame by creating a new column 'Product Name'
    and assigning names based on Product ID lookups from product_names_df.

    Works without merging — row-by-row update using a map.
    """

    # Make a copy to avoid modifying original data
    df = main_df.copy()

    # --- Normalize Product IDs for consistency ---
    df["Product ID"] = df["Product ID"].astype(str).str.strip().str.upper()
    product_names_df["Product ID"] = product_names_df["Product ID"].astype(str).str.strip().str.upper()

    # --- Auto-detect final name column from reflection results ---
    name_cols = [col for col in product_names_df.columns if col.startswith("RefinedName_")]
    if not name_cols:
        raise ValueError("❌ No 'RefinedName_X' column found in product_names_df.")
    name_col = name_cols[-1]  # use the most recent reflection pass

    # --- Build Product ID → Name mapping dictionary ---
    name_map = dict(zip(product_names_df["Product ID"], product_names_df[name_col]))

    # --- Create new column and update values based on Product ID ---
    df["Product Name"] = df["Product ID"].map(name_map)

    # --- Debug summary ---
    filled = df["Product Name"].notna().sum()
    total = len(df)
    print(f" Assigned product names to {filled}/{total} rows using Product ID mapping.")
    print(f" New column added: 'Product Name'")

    return df


In [78]:
# Testing old pipeline:

# row = updated_df.iloc[0]
# print(row)

# #  Executing Reflection pipeline for product name creation:
# for i in range(3):
#     print(f"\n=== Row {i+1} ===")
#     prod_name = multiple_pass_reflection(updated_df.iloc[i])
#     print(f"Reflection row {i+1} prod_name:", prod_name)



In [79]:
# sub_df = df[df["Product ID"] == "T0003"]
# sub_df

s = gather_product_data(df, "T0003")

print(s)


Category: Toys, Avg Price: 55.95, Avg Discount: 10.27%, Common Weather: Rainy, Common Season: Autumn, Holiday/Promotion Frequency: 373, Active Regions: West, East, South, North.


In [ ]:

# Testing new pipeline:

# Generate new product names:
df_sample = df.head(3) # Subset your dataframe
product_names_df, prod_map_val = multiple_pass_reflection_by_product(df_sample, num_passes=2)
# product_names_df, prod_map_val = multiple_pass_reflection_by_product(df, num_passes=2)



🧩 Found 100 unique products

🔹 Processing Product ID: G0001 (Regenerate: True)

📘 product_data for G0001:
Category: Groceries, Avg Price: 55.81, Avg Discount: 10.35%, Common Weather: Cloudy, Common Season: Summer, Holiday/Promotion Frequency: 356, Active Regions: North, South, West, East.

🪄 Initial Name: HarvestEssence Cloudberry Jam
💬 Reflection 1: HarvestEssence Summer Cloudberry Preserve
💬 Reflection 2: HarvestEssence Summer Cloudberry Delight

🔹 Processing Product ID: T0002 (Regenerate: True)

📘 product_data for T0002:
Category: Toys, Avg Price: 55.79, Avg Discount: 9.55%, Common Weather: Snowy, Common Season: Autumn, Holiday/Promotion Frequency: 350, Active Regions: South, West, North, East.

🪄 Initial Name: Snowy Adventure Plush Sled
💬 Reflection 1: Snowy Luxe Plush Sled
💬 Reflection 2: Snowy Luxe Plush Sledding Toy

🔹 Processing Product ID: T0003 (Regenerate: True)

📘 product_data for T0003:
Category: Toys, Avg Price: 55.95, Avg Discount: 10.27%, Common Weather: Rainy, Common S

In [84]:
product_names_df

,Product ID,Category,RefinedName_2
0,G0001,Groceries,HarvestEssence Summer Cloudberry Delight
1,T0002,Toys,Snowy Luxe Plush Sledding Toy
2,T0003,Toys,Autumn Splash Jumper Boots
3,T0004,Toys,SunnyQuest Adventure Explorer Backpack
4,E0005,Electronics,SunnyWave Luxe Winter Bluetooth Speaker
...,...,...,...
95,G0015,Groceries,FrostedHarvest Cozy Winter Soup Mix
96,F0020,Furniture,Luxe Frosted Maple Lounge Chair
97,F0001,Furniture,SunnySpring Luxe Patio Lounge Chair
98,F0016,Furniture,LuxeRain Summer Lounge Chair


In [93]:
# new_df_names = assign_product_names_to_main_df(df_sample, product_names_df)
new_df_names = assign_product_names_to_main_df(df, product_names_df)
new_df_names

✅ Assigned product names to 73100/73100 rows using Product ID mapping.
🧩 New column added: 'Product Name'


,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality,New Product ID,Product Name
0,2022-01-01,S001,G0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn,G0001,HarvestEssence Summer Cloudberry Delight
1,2022-01-01,S001,T0002,Toys,South,204,150,66,144.04,63.01,20,Sunny,0,66.16,Autumn,T0002,Snowy Luxe Plush Sledding Toy
2,2022-01-01,S001,T0003,Toys,West,102,65,51,74.02,27.99,10,Sunny,1,31.32,Summer,T0003,Autumn Splash Jumper Boots
3,2022-01-01,S001,T0004,Toys,North,469,61,164,62.18,32.72,10,Cloudy,1,34.74,Autumn,T0004,SunnyQuest Adventure Explorer Backpack
4,2022-01-01,S001,E0005,Electronics,East,166,14,135,9.26,73.64,0,Sunny,0,68.95,Summer,E0005,SunnyWave Luxe Winter Bluetooth Speaker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73095,2024-01-01,S005,F0016,Furniture,East,96,8,127,18.46,73.73,20,Snowy,0,72.45,Winter,F0016,LuxeRain Summer Lounge Chair
73096,2024-01-01,S005,T0017,Toys,North,313,51,101,48.43,82.57,10,Cloudy,0,83.78,Autumn,T0017,NimbusPlay CloudGlide Kite
73097,2024-01-01,S005,C0018,Clothing,West,278,36,151,39.65,11.11,10,Rainy,0,10.91,Winter,C0018,SnowSprout Luxe Fleece Pullover
73098,2024-01-01,S005,T0019,Toys,East,374,264,21,270.52,53.14,20,Rainy,0,55.80,Spring,T0019,CloudPlay Adventure Splash Kit


In [94]:
len(new_df_names)

73100

In [95]:

# Saving new product names xlsx:
product_name_updated_file = f"op_prod_name_ref_data_test"
save_upd_name_xlsx = save_dataframe_to_xlsx(new_df_names, product_name_updated_file)

File saved successfully → op_prod_name_ref_data_test.xlsx


## Categorywise Ads Generation

In [118]:

# -----------------------------------------------
#  Utility: Category wise context extraction
# -----------------------------------------------

def gather_category_context_with_products(df, product_names_df, category):
    """
    Aggregates contextual data for a given category (from main df) and
    appends all generated product names belonging to that category.
    """

    category = str(category).strip().title()
    sub_df = df[df["Category"].str.strip().str.title() == category].copy()
    if sub_df.empty:
        return ""

    # Category-level stats
    avg_price = round(sub_df["Price"].mean(), 2)
    avg_discount = round(sub_df["Discount"].mean(), 2)
    common_weather = sub_df["Weather Condition"].mode().iloc[0] if not sub_df["Weather Condition"].mode().empty else "Varied"
    common_season = sub_df["Seasonality"].mode().iloc[0] if not sub_df["Seasonality"].mode().empty else "All Seasons"
    holiday_freq_pct = round((sub_df["Holiday/Promotion"].sum() / len(sub_df)) * 100, 2)
    regions = ", ".join(sorted(sub_df["Region"].dropna().unique()))

    # Get all product names for this category
    product_subset = product_names_df[product_names_df["Category"].str.strip().str.title() == category]
    product_names = product_subset["RefinedName_2"].dropna().unique().tolist()
    product_list = ", ".join(product_names[:15])  # limit for prompt brevity

    context = (
        f"Category: {category}\n"
        f"Average Price: ₹{avg_price}\n"
        f"Average Discount: {avg_discount}%\n"
        f"Common Weather: {common_weather}\n"
        f"Common Season: {common_season}\n"
        f"Holiday/Promotion Frequency: {holiday_freq_pct}%\n"
        f"Active Regions: {regions}\n"
        f"Products in this Category: {product_list}"
    )
    return context


In [119]:

# -----------------------------------------------
#  Utility: Category wise Holoday Ads Generation 
# -----------------------------------------------


# def generate_autonomous_category_ads_with_products(df, product_names_df, category, num_ads=5):
#     """
#     Generates realistic ad campaigns for a category using both the
#     contextual data and actual product names from product_names_df.
#     The LLM infers campaign names, offers, and date ranges automatically.
#     """

#     category_context = gather_category_context_with_products(df, product_names_df, category)
#     if not category_context:
#         print(f"⚠️ No data found for category: {category}")
#         return []

#     prompt = f"""
#     You are a top-tier retail marketing copywriter.

#     Based on the following category and product data,
#     generate multiple realistic advertisement campaigns with seasonal or event-based offers.
#     Each campaign should include a plausible date range.

#     **Category & Product Context:**
#     {category_context}

#     Guidelines:
#     - Infer {num_ads} realistic promotional campaigns for this category.
#     - Invent short date ranges (e.g., “Dec 15 – Jan 05”, “Jul 01 – Jul 15”) matching the tone of the season.
#     - Each advertisement should:
#         • Mention real product names naturally in the copy.
#         • Include an offer (e.g., “up to 40% off” or “Buy 2 Get 1 Free”).
#         • Capture the seasonal tone (festive, fresh, cozy, etc.)
#         • Be 40–90 words, compelling and professional.
#         • Optionally include a CTA (Shop now, Don’t miss out, Limited time offer, etc.)
#     - Output format:
#         1. [Campaign Name] – [Date Range]: [Advertisement Text]
#         2. ...

#     Output only the numbered list.
#     """

#     result = llm.invoke([
#         SystemMessage(content="You are a professional retail marketing strategist."),
#         HumanMessage(content=prompt)
#     ])
#     raw_output = result.content.strip()

#     # Parse structured LLM output
#     ads = []
#     for line in raw_output.split("\n"):
#         if line.strip() and line[0].isdigit():
#             try:
#                 campaign_part, ad_text = line.split(":", 1)
#                 if "–" in campaign_part:
#                     campaign_name, date_range = campaign_part.split("–", 1)
#                 elif "-" in campaign_part:
#                     campaign_name, date_range = campaign_part.split("-", 1)
#                 else:
#                     campaign_name, date_range = campaign_part, ""
                    
#                 ads.append({
#                     "Category": category,
#                     "Campaign": campaign_name.strip(" ."),
#                     "Date Range": date_range.strip(),
#                     "Advertisement": ad_text.strip()
#                 })
#             except ValueError:
#                 continue

#     print(f"✅ Generated {len(ads)} ads for category: {category}")
#     return ads



def generate_single_category_ads(df, product_names_df, category, num_ads=5, max_retries=3):
    """
    Generates a specified number of advertisements for a single category.
    Uses global LLM instance to infer seasonal date ranges, offers, and ad copy automatically.
    Retries if the output is incomplete.
    """

    category_context = gather_category_context_with_products(df, product_names_df, category)
    if not category_context:
        print(f"⚠️ No data found for category: {category}")
        return []

    def _generate_once():
        prompt = f"""
        You are a professional retail marketing strategist and copywriter.

        Based on the following category and product data, create exactly **{num_ads}**
        seasonal or event-based advertisement campaigns.

        **Category & Product Context:**
        {category_context}

        Each campaign must include:
        - Campaign Name
        - Date Range (realistic, e.g., "Dec 10 – Dec 30" or "Jul 01 – Jul 15")
        - Advertisement Text (40–90 words)
        - Natural inclusion of real product names
        - An offer ("up to X% off", "Buy 2 Get 1 Free", etc.)
        - Optional CTA ("Shop now!", "Limited time offer!", etc.)

        The tone should fit the season or context (festive, cozy, refreshing, etc.)
        Avoid repetition between campaigns.

        Output format (exactly {num_ads} items):
        1. [Campaign Name] – [Date Range]: [Advertisement Text]
        2. ...
        """

        result = llm.invoke([
            SystemMessage(content="You are a disciplined and creative retail marketing copywriter."),
            HumanMessage(content=prompt)
        ])
        return result.content.strip()

    retries = 0
    ads = []
    while retries < max_retries:
        raw_output = _generate_once()

        parsed_ads = []
        for match in re.finditer(r"(\d+)\.\s*(.*?)\s*[-–]\s*(.*?):\s*(.*)", raw_output):
            try:
                campaign_name = match.group(2).strip()
                date_range = match.group(3).strip()
                text = match.group(4).strip()
                parsed_ads.append({
                    "Category": category,
                    "Campaign": campaign_name,
                    "Date Range": date_range,
                    "Advertisement": text
                })
            except Exception:
                continue

        ads = parsed_ads
        if len(ads) >= num_ads:
            break

        retries += 1
        print(f"⚠️ [{category}] Only got {len(ads)} ads (target {num_ads}), retrying {retries}/{max_retries}...")
        time.sleep(2)

    print(f"✅ [{category}] Generated {len(ads)} ads successfully.")
    return ads



def generate_all_category_ads(df, product_names_df, num_ads_per_category=5):
    """
    Generates multiple ads for all unique categories found in product_names_df.
    Returns a combined DataFrame only — does not save to disk.
    """

    all_ads = []
    categories = sorted(product_names_df["Category"].dropna().unique().tolist())
    print(f"\n🧩 Found {len(categories)} unique categories.")

    for cat in categories:
        print(f"\n🪧 Generating {num_ads_per_category} ads for category: {cat}")
        ads = generate_single_category_ads(
            df=df,
            product_names_df=product_names_df,
            category=cat,
            num_ads=num_ads_per_category
        )
        all_ads.extend(ads)

    ads_df = pd.DataFrame(all_ads)
    print(f"\n📊 Total ads generated: {len(ads_df)} across {len(categories)} categories.")
    return ads_df



In [122]:
# Generate all ads
ads_df = generate_all_category_ads(
    df=df,
    product_names_df=product_names_df,
    num_ads_per_category=12
)



🧩 Found 5 unique categories.

🪧 Generating 12 ads for category: Clothing
✅ [Clothing] Generated 12 ads successfully.

🪧 Generating 12 ads for category: Electronics
✅ [Electronics] Generated 12 ads successfully.

🪧 Generating 12 ads for category: Furniture
✅ [Furniture] Generated 12 ads successfully.

🪧 Generating 12 ads for category: Groceries
✅ [Groceries] Generated 12 ads successfully.

🪧 Generating 12 ads for category: Toys
✅ [Toys] Generated 12 ads successfully.

📊 Total ads generated: 60 across 5 categories.


In [123]:
# Inspect ads
ads_df.head()

,Category,Campaign,Date Range,Advertisement
0,Clothing,**Winter Wonderland Sale**,Dec 01 – Dec 15,Embrace the chill with our Winter Wonderland S...
1,Clothing,**Festive Fashion Frenzy**,Dec 16 – Dec 26,Celebrate the season with our Festive Fashion ...
2,Clothing,"**New Year, New Wardrobe**",Dec 27 – Jan 10,Kickstart the New Year with a fresh wardrobe! ...
3,Clothing,**Cozy Comforts Collection**,Jan 11 – Jan 25,Dive into the Cozy Comforts Collection! Snuggl...
4,Clothing,**Spring Refresh Sale**,Feb 01 – Feb 14,Welcome spring with open arms! Refresh your wa...


In [124]:
# Save ads:
final_ads_file_name = f"category_ads_with_products_2"
final_save = save_dataframe_to_xlsx(ads_df, final_ads_file_name)
print(final_save)

File saved successfully → category_ads_with_products_2.xlsx
category_ads_with_products_2.xlsx


In [125]:
ads_df.iloc[0]["Advertisement"]

'Embrace the chill with our Winter Wonderland Sale! Wrap yourself in style with the Luxe Cloudspire Winter Wrap Coat or the Luxe Cozy Autumn Cardigan. Enjoy a warm discount of up to 20% off on selected items. Don’t miss this chance to elevate your winter wardrobe! Shop now!'

In [142]:
ads_df["holiday_ad_content"] = ads_df.apply( lambda row: f"{row['Campaign']}\t{row['Date Range']}\n{row['Advertisement']}", axis=1 )

print(ads_df.iloc[10])
	
    


Category                                                       Clothing
Campaign                                       **Back to School Style**
Date Range                                              Jul 01 – Jul 15
Advertisement         Gear up for a stylish school year! Discover co...
holiday_ad_content    **Back to School Style**\tJul 01 – Jul 15\nGea...
Name: 10, dtype: object


In [145]:
final_ads_file_name = f"final_categ_ads_products_data"  # f"category_ads_with_products_3"
final_save = save_dataframe_to_xlsx(ads_df, final_ads_file_name)
print(final_save)


File saved successfully → final_categ_ads_products_data.xlsx
final_categ_ads_products_data.xlsx


In [146]:
print(ads_df.iloc[2])


Category                                                       Clothing
Campaign                                     **New Year, New Wardrobe**
Date Range                                              Dec 27 – Jan 10
Advertisement         Kickstart the New Year with a fresh wardrobe! ...
holiday_ad_content    **New Year, New Wardrobe**\tDec 27 – Jan 10\nK...
Name: 2, dtype: object
